micro K8s tutorial via K3s. We could use minikube for it as well, but it does not let you spawn multiple nodes, which sucks

Clone docker-compose from https://github.com/rancher/k3s/blob/master/docker-compose.yml

In [1]:
%%bash

docker-compose up -d --scale node=3

Creating network "k8s_default" with the default driver
Creating volume "k8s_k3s-server" with default driver
Creating k8s_server_1 ... 
Creating k8s_node_1   ... 
Creating k8s_node_2   ... 
Creating k8s_node_3   ... 


In [2]:
!docker-compose ps

    Name                 Command             State              Ports           
--------------------------------------------------------------------------------
k8s_node_1     /bin/k3s agent                Up                                 
k8s_node_2     /bin/k3s agent                Up                                 
k8s_node_3     /bin/k3s agent                Up                                 
k8s_server_1   /bin/k3s server --disable-    Up      0.0.0.0:6443->6443/tcp,    
               ...                                   0.0.0.0:8888->80/tcp       


Copy kube config so that we could login


In [9]:
!cp kubeconfig.yaml ~/.kube/config

Try connection:

should print 3 nodes. If you have connection error, run above command again. There might be a delay


In [10]:
!kubectl get nodes

NAME           STATUS    ROLES     AGE       VERSION
8e6e3de3af5d   Ready     <none>    44s       v1.14.1-k3s.4
cde3bc10967e   Ready     <none>    44s       v1.14.1-k3s.4
f1119b2d4b3e   Ready     <none>    44s       v1.14.1-k3s.4


In [11]:
#should be empty
!kubectl get pods

No resources found.


In [12]:
# this will deploy 2 a simple hello-world app
!kubectl run hello-world \
    --replicas=2 \
    --labels="run=load-balancer-example" \
    --image=gcr.io/google-samples/node-hello:1.0  \
    --port=8080

deployment.apps "hello-world" created


In [26]:
!kubectl get pods

NAME                          READY     STATUS    RESTARTS   AGE
hello-world-68ff65cf7-bf8hq   1/1       Running   0          16m
hello-world-68ff65cf7-gbs6t   1/1       Running   0          16m


In [27]:
# hIf you network is slow enough you'll see availeble: 0

!kubectl get deployments

NAME          DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
hello-world   2         2         2            2           16m


In [28]:
# ere will be detailed info on each pod
!kubectl describe pods

Name:               hello-world-68ff65cf7-bf8hq
Namespace:          default
Priority:           0
PriorityClassName:  <none>
Node:               f1119b2d4b3e/172.25.0.4
Start Time:         Sat, 15 Jun 2019 23:09:27 +0200
Labels:             pod-template-hash=68ff65cf7
                    run=load-balancer-example
Annotations:        <none>
Status:             Running
IP:                 10.42.2.3
Controlled By:      ReplicaSet/hello-world-68ff65cf7
Containers:
  hello-world:
    Container ID:   containerd://fb57d56f5739307ba672c88efb705c532f2ef2d8483fc04a02b3dee570199bc8
    Image:          gcr.io/google-samples/node-hello:1.0
    Image ID:       sha256:9ef4b4c241fc28ce894a57a7cb218b796838fdccf15515d784791abc2ffc1ee6
    Port:           8080/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Sat, 15 Jun 2019 23:17:04 +0200
    Ready:          True
    Restart Count:  0
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount 

Pods are like docker instances. They have lifetime, and no orchestration. To expose ports to outworld we have to create a service.

In [32]:
# at this point we don't have any service
!kubectl describe services

Name:              kubernetes
Namespace:         default
Labels:            component=apiserver
                   provider=kubernetes
Annotations:       <none>
Selector:          <none>
Type:              ClusterIP
IP:                10.43.0.1
Port:              https  443/TCP
TargetPort:        6445/TCP
Endpoints:         127.0.0.1:6445
Session Affinity:  None
Events:            <none>


Expose a deployemnt for pods with name 'example-service'.
Simplest networking type would be NodePort, which will expose port on a public IP of each nodee. Not very usefull but enough for the start

In [33]:
!kubectl expose deployment hello-world \
    --type=NodePort \
    --name=example-service

service "example-service" exposed


In [34]:
!kubectl describe services

Name:                     example-service
Namespace:                default
Labels:                   run=load-balancer-example
Annotations:              <none>
Selector:                 run=load-balancer-example
Type:                     NodePort
IP:                       10.43.192.14
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  32346/TCP
Endpoints:                10.42.0.2:8080,10.42.2.3:8080
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


Name:              kubernetes
Namespace:         default
Labels:            component=apiserver
                   provider=kubernetes
Annotations:       <none>
Selector:          <none>
Type:              ClusterIP
IP:                10.43.0.1
Port:              https  443/TCP
TargetPort:        6445/TCP
Endpoints:         127.0.0.1:6445
Session Affinity:  None
Events:            <none>


In [35]:
# load pods with public IPs
!kubectl get pods --selector="run=load-balancer-example" --output=wide

NAME                          READY     STATUS    RESTARTS   AGE       IP          NODE
hello-world-68ff65cf7-bf8hq   1/1       Running   0          20m       10.42.2.3   f1119b2d4b3e
hello-world-68ff65cf7-gbs6t   1/1       Running   0          20m       10.42.0.2   8e6e3de3af5d


take an ip from output and use in the following command. We use docker-compose and have no access to the internals of nodes. 

In [37]:
!docker exec -t k8s_node_1 wget -qO- 10.42.2.3:8080

Hello Kubernetes!

If you see "Hello Kubernetes", then it works. Another hack would be to use `kubectl port-forward {podName}` command which would run a proxy to the pod

Cleaning up: Graceful service deletion

In [38]:
!kubectl delete service example-service

service "example-service" deleted


# Exterminatus !

cleanup docker-compose and k8s cluster with all of it information and state

In [39]:
!docker-compose down
!docker volume rm k8s_k3s-server


Stopping k8s_node_2   ... 
Stopping k8s_node_3   ... 
Stopping k8s_node_1   ... 
Stopping k8s_server_1 ... 
ping k8s_server_1 ... doneRemoving k8s_node_2   ... 
Removing k8s_node_3   ... 
Removing k8s_node_1   ... 
Removing k8s_server_1 ... 
ving k8s_node_1   ... doneRemoving network k8s_default
k8s_k3s-server
